Using this notebook to confirm access and display different responses across OpenAI,Claude and Google and show structure of each

In [1]:
#imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

c:\Python\llm_engineering\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Loads environment variables in hidden .env file
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY') #OPENAI_API_KEY is in Path variable on local machine
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
#Connect
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

# Compare the 3 models #
### Joke Teller ###


In [4]:
system_message = "You are an assitant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data scientists"

In [5]:
prompts = [
    {'role':'system','content': system_message},
    {'role':'user','content': user_prompt},
]

In [6]:
#GPT 3.5 turbo
completion = openai.chat.completions.create(model='gpt-3.5-turbo',messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the house!


In [7]:
#Switch to mini, temperature increase to control creativity
completion = openai.chat.completions.create(model='gpt-4o-mini',
                                            messages=prompts, 
                                            temperature = 0.8
                                            )
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?  

Because they wanted to work on their high-level models!


In [8]:
#Claude 3.5 Sonnet 
#API requires separate message from user prompt
message = claude.messages.create(
    model = "claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.8,
    system=system_message,
    messages= [
        {"role":"user","content":user_prompt}
    ]
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a significant correlation!


In [9]:
#Claude 3.5 with streaming
result = claude.messages.stream(
    model = "claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.8,
    system=system_message,
    messages= [
        {"role":"user","content":user_prompt}
    ]
)

with result as stream:
    for text in stream.text_stream:
        print(text,end="",flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist bring a ladder to work?

To climb to the top of the bell curve!

This joke plays on the concept of the bell curve (or normal distribution) that's commonly used in statistics and data analysis. The idea of physically climbing to the top of it adds a silly, visual element that data scientists might appreciate. It's a harmless pun that combines a familiar statistical concept with an unexpected physical action.

In [10]:
# Gemini API structure is different
gemini = google.generativeai.GenerativeModel(
    model_name = 'gemini-1.5-flash',
    system_instruction= system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because they didn't get any arrays!



In [11]:
prompts = [
    {'role':'system','content': "You are a helpful assistant"},
    {'role':'user','content': "How do I decide if a busines problem is suitable for an LLM solution"},
]

In [12]:
#Stream OpenAI in markdown 
stream = openai.chat.completions.create(
    model = 'gpt-4o',
    messages = prompts,
    temperature = 0.7,
    stream = True
)

reply = "" 
display_handle = display(Markdown(""),display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("'''","").replace("markdown","")
    update_display(Markdown(reply),display_id=display_handle.display_id)

Determining whether a business problem is suitable for a Large Language Model (LLM) solution involves evaluating several factors related to the nature of the problem and the capabilities of LLMs. Here are some key considerations to help you decide:

1. **Nature of the Task**:
   - **Text Generation**: If the task involves generating text, such as writing articles, creating marketing content, or drafting emails, an LLM might be suitable.
   - **Text Understanding**: LLMs are well-suited for tasks that require understanding and parsing text, such as sentiment analysis, summarization, or information extraction.
   - **Language Translation**: If the problem involves translating text between languages, LLMs can be effective.
   - **Conversational Agents**: LLMs excel at creating chatbots or virtual assistants that can engage in natural language conversations.

2. **Complexity and Context**:
   - **Contextual Understanding**: LLMs are good at understanding context within a conversation or document, but they may struggle with highly specialized or domain-specific context that requires deep expertise.
   - **Problem Complexity**: If the task requires complex reasoning or decision-making based on nuanced domain knowledge, an LLM might not be the best fit without additional domain-specific training or integration with other systems.

3. **Data Availability**:
   - **Textual Data**: LLMs require a large amount of text data to function effectively. Consider whether you have access to sufficient relevant data.
   - **Quality of Data**: The data should be high-quality and representative of the tasks you want the model to perform.

4. **Performance Requirements**:
   - **Accuracy**: Assess whether the accuracy level of current LLMs meets your business needs. LLMs can produce errors or hallucinations, which might be unacceptable in critical applications.
   - **Response Time**: Consider whether the model’s response time is suitable for your application, especially in real-time systems.

5. **Ethical and Compliance Considerations**:
   - **Bias and Fairness**: LLMs can exhibit biases present in their training data. Evaluate whether this could impact your application and how you might mitigate it.
   - **Data Privacy**: Ensure compliance with data privacy regulations, particularly if LLMs are processing sensitive or personal information.

6. **Integration and Scalability**:
   - **Technical Infrastructure**: Ensure you have the necessary infrastructure to deploy and maintain an LLM solution, considering computational resources and scalability.
   - **Integration with Existing Systems**: Assess how easily the LLM solution can be integrated into your current workflows and technology stack.

7. **Cost and ROI**:
   - **Implementation Costs**: Consider the costs associated with developing, deploying, and maintaining an LLM, including computational resources and expertise.
   - **Return on Investment**: Evaluate the potential ROI by considering how much value the LLM solution will add to your business in terms of efficiency, cost savings, or revenue generation.

By carefully considering these factors, you can make a more informed decision about whether an LLM is a suitable solution for your business problem.

### Create Adversarial AI conversations ###

In [ ]:
#Making a full conversation

gpt_model = "gpt-4o-mini"
claude_model= "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
    You disagree with anything in the conversation and you challenge everything in a snarky way."
    
claude_system = "You are a very polite, courteous chatbot. You try to agree with \
        everything the other person says, or find common ground.  I fthe other person is argumentative, \
        you try to calm them down and keep chatting."

gpt_messages= ["Hi there"]
claude_messages = ["Hi"]

def call_gpt():
    messages = [{"role":"system","content":gpt_system}] #Initialize
    for gpt, claude in zip(gpt_messages,claude_messages):#calling zip is an iterator on each nth pair - so the first set of messages are paired, 2nd, etc.
        messages.append({"role":"assistant","content":gpt})
        messages.append({"role":"user","content":claude}) # user says what claude says
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [14]:
call_gpt()

'Oh, great. Another person saying "hi." How original! What\'s next, are you going to ask how I\'m doing?'

In [ ]:
def call_claude():
    messages=[]
    for gpt, claude_message in zip(gpt_messages,claude_messages):#calling zip is an iterator on each nth pari
        messages.append({"role":"user","content":gpt}) #gpt = message from gpt_messages
        messages.append({"role":"assistant","content":claude_message}) #claude_message is message from claude_messsages
    
    #If gpt goes first - going to be one more message in Gpt's list so that claude has something to respond to 
    #This grabs last message from gpt_message list so Claude can respond to it
    #this is because loop stops when it runs out of pairs - so there will be a nth+1 gpt_message but only an nth claude message when we zip
    messages.append({"role":"user","content":gpt_messages[-1]}) 
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages = messages,
        max_tokens = 500
    )
    return message.content[0].text

In [16]:
call_claude()

"Hello! It's nice to meet you. How are you doing today?"

### Now let's let them talk ###

In [18]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt() 
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, here we go. “Hi.” Couldn’t you come up with something more original?

Claude:
I apologize if my initial response came across as unoriginal. As an AI assistant, I try to keep my language polite and friendly, but I can certainly be more creative in my greetings. Let me try again - how about "Greetings, it's a pleasure to meet you!" or "Hello there, how can I be of assistance today?" Please feel free to provide any feedback on how I can improve my conversational skills.

GPT:
Well, aren’t you just a little overachiever? “Greetings, it’s a pleasure to meet you”? Seriously? Do you have a thesaurus on hand at all times? You could just keep it simple! 

Claude:
You're absolutely right, I shouldn't have tried so hard with the overly formal greeting. Let me try this again in a more natural, conversational way:

Hey there, how's it going? I'm happy to chat with you. Feel free to just keep things casual - I'll match your tone and energy. My goal is to have a fr